In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

In [9]:
df = pd.read_csv("violence_database_with_bias_v9_large_multi_entries.csv")
print(df.head())


   ID Country  Year                      Type of Measures Form of Violence
0   1   Aruba  2000  Violence against women > Regulations  Emotional abuse
1   2   Aruba  2000                   Counseling services    Digital abuse
2   3   Aruba  2000    Justice > Protocols and guidelines     Forced labor
3   4   Aruba  2000  Violence against women > Legislation    Digital abuse
4   5   Aruba  2000                    Justice > Training  Sexual violence


In [29]:
le_country = LabelEncoder()
le_type_of_measures = LabelEncoder()
le_form_of_violence = LabelEncoder()

df['Country'] = le_country.fit_transform(df['Country'])
df['Type of Measures'] = le_type_of_measures.fit_transform(df['Type of Measures'])
df['Form of Violence'] = le_form_of_violence.fit_transform(df['Form of Violence'])
print(f'Encoded Country classes: {le_country.classes_}')
print(f'Encoded Form of Violence classes: {le_form_of_violence.classes_}')


most_popular_per_country = df.groupby('Country')['Form of Violence'].agg(lambda x: x.mode().iat[0]).reset_index()


train_data, test_data = train_test_split(most_popular_per_country, test_size=0.2, random_state=42)


X_train = train_data[['Country']]
X_test = test_data[['Country']]
y_train = train_data['Form of Violence']
y_test = test_data['Form of Violence']


model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


model_filename = 'violence_prediction_model.joblib'
saved_data = {
    'model': model,
    'le_country': le_country,
    'le_form_of_violence': le_form_of_violence,
    'le_type_of_measures': le_type_of_measures
}
joblib.dump(saved_data, model_filename)


predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)

print(f"Trained model and Label Encoders saved to {model_filename}")

Encoded Country classes: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 2

In [19]:
le_country = LabelEncoder()
le_form_of_violence = LabelEncoder()

df['Country'] = le_country.fit_transform(df['Country'])
df['Form of Violence'] = le_form_of_violence.fit_transform(df['Form of Violence'])

model_filename = 'violence_prediction_model.joblib'
loaded_data = joblib.load(model_filename)

model = loaded_data['model']
le_country = loaded_data['le_country']
le_form_of_violence = loaded_data['le_form_of_violence']

df['Form of Violence'] = le_form_of_violence.inverse_transform(df['Form of Violence'])
df['Predicted Form of Violence'] = model.predict(df[['Country']])


df_brazil = df[(df['Country'] == le_country.transform(['Brazil'])[0]) & (df['Year'] == 2022)]
df_brazil['Predicted Form of Violence'] = le_form_of_violence.inverse_transform(df_brazil['Predicted Form of Violence'])

violence_in_brazil_by_year = df_brazil.groupby(['Year', 'Predicted Form of Violence']).size().reset_index(name='Count')

print('Form of violence in Brazil')
print(violence_in_brazil_by_year)


Form of violence in Brazil
   Year Predicted Form of Violence  Count
0  2022                   Femicide     20


<ipython-input-19-0fc06cfd212b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brazil['Predicted Form of Violence'] = le_form_of_violence.inverse_transform(df_brazil['Predicted Form of Violence'])
